In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, KMeans
import plotly.express as px

# Step 1: Load and Preprocess the Data
file_path = 'cleaned_electricity_data.csv'  n  
df_clean = pd.read_csv(file_path, parse_dates=['Timestamp'])

# Pivoting the data to have customers as rows and timestamps as features
df_pivot = df_clean.pivot_table(index='Customer_ID', columns='Timestamp', values='Consumption_kWh', aggfunc='sum')

# Fill missing values with zeros
df_pivot.fillna(0, inplace=True)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_pivot)

ImportError: Traceback (most recent call last):
  File "c:\Users\khiz2\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [2]:
# Step 2: Build the Autoencoder
input_dim = scaled_data.shape[1]
encoding_dim = 32  # Dimension of the compressed space

# Encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# Decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Autoencoder Model
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

NameError: name 'scaled_data' is not defined

In [ ]:
# Step 3: Train the Autoencoder
autoencoder.fit(scaled_data, scaled_data, epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

# Extract the compressed representation
encoder = Model(inputs=input_layer, outputs=encoded)
compressed_data = encoder.predict(scaled_data)

In [ ]:
# Step 4: Apply Clustering (DBSCAN)
dbscan = DBSCAN(eps=1.0, min_samples=5)
cluster_labels = dbscan.fit_predict(compressed_data)

In [ ]:
# Step 5: Visualize Clusters
pca_df = pd.DataFrame(compressed_data, columns=[f'Feature_{i+1}' for i in range(encoding_dim)])
pca_df['Customer_ID'] = df_pivot.index
pca_df['Cluster'] = cluster_labels

fig = px.scatter(
    pca_df, x='Feature_1', y='Feature_2',
    color='Cluster',
    hover_data=['Customer_ID'],
    title='Autoencoder + DBSCAN Clustering of Customer Consumption Patterns',
    color_continuous_scale='Viridis'
)

fig.update_traces(marker=dict(size=10, opacity=0.7, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    title_x=0.5,
    hovermode='closest',
    template='plotly_dark',
    xaxis_title='Compressed Feature 1',
    yaxis_title='Compressed Feature 2'
)

fig.show()

In [ ]:
print("Cluster Distribution:\n", pca_df['Cluster'].value_counts())